# Load Data

In [7]:
import pandas as pd

data = pd.read_csv('data_processed', encoding="latin-1")
data.head()

,Unnamed: 0,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,...,rms_mean,rms_std,spectral_centroid_mean,spectral_centroid_std,spectral_bandwidth_mean,spectral_bandwidth_std,spectral_flatness_mean,spectral_flatness_std,tempogram_mean,tempogram_std
0,0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,...,0.034636,0.058105,5335.603726,2688.930490,4615.955290,1007.859698,0.092827,0.133516,0.239535,0.137067
1,1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,...,0.030456,0.063953,6338.191486,3683.342441,4989.509117,1157.665970,0.142638,0.155706,0.139604,0.163539
2,2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,...,0.037484,0.067771,6077.338238,3792.722720,4680.404598,1377.311199,0.124579,0.146646,0.179192,0.130047
3,3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,...,0.022330,0.049949,5542.225095,3417.495525,4875.089779,1265.250954,0.106046,0.138443,0.208746,0.174764
4,4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,...,0.021056,0.051182,7714.574395,3887.263988,5119.254721,1091.703500,0.188627,0.157474,0.129188,0.149107


In [15]:
data = data.drop(['Unnamed: 0'], axis=1)
data.head()

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,...,rms_mean,rms_std,spectral_centroid_mean,spectral_centroid_std,spectral_bandwidth_mean,spectral_bandwidth_std,spectral_flatness_mean,spectral_flatness_std,tempogram_mean,tempogram_std
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,...,0.034636,0.058105,5335.603726,2688.930490,4615.955290,1007.859698,0.092827,0.133516,0.239535,0.137067
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,...,0.030456,0.063953,6338.191486,3683.342441,4989.509117,1157.665970,0.142638,0.155706,0.139604,0.163539
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,...,0.037484,0.067771,6077.338238,3792.722720,4680.404598,1377.311199,0.124579,0.146646,0.179192,0.130047
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,...,0.022330,0.049949,5542.225095,3417.495525,4875.089779,1265.250954,0.106046,0.138443,0.208746,0.174764
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,...,0.021056,0.051182,7714.574395,3887.263988,5119.254721,1091.703500,0.188627,0.157474,0.129188,0.149107


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               1090 non-null   int64  
 1   drummer                  1090 non-null   object 
 2   session                  1090 non-null   object 
 3   id                       1090 non-null   object 
 4   style                    1090 non-null   object 
 5   bpm                      1090 non-null   int64  
 6   beat_type                1090 non-null   object 
 7   time_signature           1090 non-null   object 
 8   midi_filename            1090 non-null   object 
 9   audio_filename           1090 non-null   object 
 10  duration                 1090 non-null   float64
 11  split                    1090 non-null   object 
 12  onset_env_mean           1090 non-null   float64
 13  onset_env_std            1090 non-null   float64
 14  mfcc_mean               

# Split labels and features

In [17]:
data.iloc[:, 11:]

,onset_env_mean,onset_env_std,mfcc_mean,mfcc_std,spectral_flux_mean,spectral_flux_std,spectral_contrast_mean,spectral_contrast_std,tonnetz_mean,tonnetz_std,rms_mean,rms_std,spectral_centroid_mean,spectral_centroid_std,spectral_bandwidth_mean,spectral_bandwidth_std,spectral_flatness_mean,spectral_flatness_std,tempogram_mean,tempogram_std
0,0.820465,1.471963,-14.319676,73.690796,0.370578,0.803874,10.217504,7.141570,-0.003644,0.048117,0.034636,0.058105,5335.603726,2688.930490,4615.955290,1007.859698,0.092827,0.133516,0.239535,0.137067
1,0.673516,1.996858,-18.866692,81.949180,0.318956,0.836129,9.389001,5.825721,0.000954,0.049296,0.030456,0.063953,6338.191486,3683.342441,4989.509117,1157.665970,0.142638,0.155706,0.139604,0.163539
2,0.807532,1.771858,-15.552315,76.598114,0.430627,0.986051,9.708105,6.514865,-0.000462,0.032169,0.037484,0.067771,6077.338238,3792.722720,4680.404598,1377.311199,0.124579,0.146646,0.179192,0.130047
3,0.714612,1.739458,-18.966784,91.778990,0.261665,0.707281,9.359258,6.028078,0.001330,0.041105,0.022330,0.049949,5542.225095,3417.495525,4875.089779,1265.250954,0.106046,0.138443,0.208746,0.174764
4,0.617521,1.897261,-21.551086,84.417250,0.230317,0.718205,9.491370,5.857463,0.002939,0.038458,0.021056,0.051182,7714.574395,3887.263988,5119.254721,1091.703500,0.188627,0.157474,0.129188,0.149107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,0.700411,1.255718,-17.177755,80.723300,0.171760,0.367869,11.095754,7.548901,-0.000483,0.038268,0.017097,0.026517,4798.587353,2262.243523,4202.586394,1143.580021,0.054484,0.086355,0.244154,0.158753
1086,0.673706,1.405090,-22.172949,84.791030,0.103791,0.292991,11.249964,7.552028,-0.000548,0.034618,0.010296,0.022294,5685.170684,1819.749391,4524.244019,929.885425,0.071707,0.090014,0.169466,0.136667
1087,0.807790,1.839772,-17.393496,84.921295,0.210763,0.417661,9.719207,6.145140,0.001560,0.037345,0.018970,0.029546,7167.742409,3295.887907,5292.859346,1174.628306,0.148190,0.145533,0.206317,0.176834
1088,0.851409,1.907641,-15.513251,79.025314,0.232315,0.420945,9.661610,6.337121,-0.000522,0.032503,0.022295,0.030756,6522.843388,3378.381352,5077.864282,1256.245421,0.144691,0.153425,0.202719,0.160704


In [16]:
data.columns

Index(['drummer', 'session', 'id', 'style', 'bpm', 'beat_type',
       'time_signature', 'midi_filename', 'audio_filename', 'duration',
       'split', 'onset_env_mean', 'onset_env_std', 'mfcc_mean', 'mfcc_std',
       'spectral_flux_mean', 'spectral_flux_std', 'spectral_contrast_mean',
       'spectral_contrast_std', 'tonnetz_mean', 'tonnetz_std', 'rms_mean',
       'rms_std', 'spectral_centroid_mean', 'spectral_centroid_std',
       'spectral_bandwidth_mean', 'spectral_bandwidth_std',
       'spectral_flatness_mean', 'spectral_flatness_std', 'tempogram_mean',
       'tempogram_std'],
      dtype='object')

In [9]:
# seperate labels
y = data.iloc[:, 11:]
# seperate data
X = data.iloc[:, :11]
# drop unnecessary data
X = X.drop(columns=["drummer", "session", "id", "midi_filename", "audio_filename", "style"])
# time signature would be a nice feature, but they are 99% in 4/4. We might as well drop this column to pevent overfitting
X = X.drop(columns=["time_signature"])

In [10]:
# only labels
y.head()

,split,onset_env_mean,onset_env_std,mfcc_mean,mfcc_std,spectral_flux_mean,spectral_flux_std,spectral_contrast_mean,spectral_contrast_std,tonnetz_mean,...,rms_mean,rms_std,spectral_centroid_mean,spectral_centroid_std,spectral_bandwidth_mean,spectral_bandwidth_std,spectral_flatness_mean,spectral_flatness_std,tempogram_mean,tempogram_std
0,test,0.820465,1.471963,-14.319676,73.690796,0.370578,0.803874,10.217504,7.141570,-0.003644,...,0.034636,0.058105,5335.603726,2688.930490,4615.955290,1007.859698,0.092827,0.133516,0.239535,0.137067
1,test,0.673516,1.996858,-18.866692,81.949180,0.318956,0.836129,9.389001,5.825721,0.000954,...,0.030456,0.063953,6338.191486,3683.342441,4989.509117,1157.665970,0.142638,0.155706,0.139604,0.163539
2,test,0.807532,1.771858,-15.552315,76.598114,0.430627,0.986051,9.708105,6.514865,-0.000462,...,0.037484,0.067771,6077.338238,3792.722720,4680.404598,1377.311199,0.124579,0.146646,0.179192,0.130047
3,test,0.714612,1.739458,-18.966784,91.778990,0.261665,0.707281,9.359258,6.028078,0.001330,...,0.022330,0.049949,5542.225095,3417.495525,4875.089779,1265.250954,0.106046,0.138443,0.208746,0.174764
4,test,0.617521,1.897261,-21.551086,84.417250,0.230317,0.718205,9.491370,5.857463,0.002939,...,0.021056,0.051182,7714.574395,3887.263988,5119.254721,1091.703500,0.188627,0.157474,0.129188,0.149107


In [11]:
# only features
X.head()

,Unnamed: 0,bpm,beat_type,duration
0,0,138,beat,27.872308
1,1,102,beat,37.691158
2,2,105,beat,36.351218
3,3,86,beat,44.716543
4,4,80,beat,47.987500


In [12]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convolutional Neural Network

In [1]:
# Import data generator from keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator